In [ ]:
!pip install river

# 1. Módulo `stream` - encapsular datasets
Permite encapsular diferentes fontes de datasets comuns e utilizar no formato de fluxo de dados, iterarando a cada instância.
Nesse exemplo usei o `stream.iter_sklearn_dataset`

In [2]:
from sklearn import datasets
from river import stream

In [3]:
diabetes = datasets.load_diabetes() # dataset do sklearn a ser encapsulado

In [6]:
k  =0
for x,y in stream.iter_sklearn_dataset(diabetes): # o método cria um generator
  print(x)
  print(y)
  k +=1
  if k==5:
    break

{'age': 0.038075906433423026, 'sex': 0.05068011873981862, 'bmi': 0.061696206518683294, 'bp': 0.0218723855140367, 's1': -0.04422349842444599, 's2': -0.03482076283769895, 's3': -0.04340084565202491, 's4': -0.002592261998183278, 's5': 0.019907486170462722, 's6': -0.01764612515980379}
151.0
{'age': -0.0018820165277906047, 'sex': -0.044641636506989144, 'bmi': -0.051474061238800654, 'bp': -0.02632752814785296, 's1': -0.008448724111216851, 's2': -0.019163339748222204, 's3': 0.07441156407875721, 's4': -0.03949338287409329, 's5': -0.0683315470939731, 's6': -0.092204049626824}
75.0
{'age': 0.08529890629667548, 'sex': 0.05068011873981862, 'bmi': 0.04445121333659049, 'bp': -0.00567042229275739, 's1': -0.04559945128264711, 's2': -0.03419446591411989, 's3': -0.03235593223976409, 's4': -0.002592261998183278, 's5': 0.002861309289833047, 's6': -0.025930338989472702}
141.0
{'age': -0.0890629393522567, 'sex': -0.044641636506989144, 'bmi': -0.011595014505211082, 'bp': -0.03665608107540074, 's1': 0.0121905

### 1.1 Exemplo mais genérico: utilizando o repositório UCI
O método `iter_pandas` adapta o dataset para o tipo adequado utilizado na river (gerador).

In [7]:
!pip install ucimlrepo

In [8]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
X = iris.data.features
y = iris.data.targets["class"]

In [9]:
for x,yi in stream.iter_pandas(X,y): # simulando fluxo de dados
  print(x)
  print(yi)
  break

{'sepal length': 5.1, 'sepal width': 3.5, 'petal length': 1.4, 'petal width': 0.2}
Iris-setosa


# 2. Treinamento básico (sem tratamento de dados)
Exemplos de treinamento sem utilizar os pipelines de tratamento de dados

## 2.1 Treinar modelo simples de regressão
Nesse exemplo o tratamento nos dados não foi feito para manter simples

In [10]:
from river.tree import HoeffdingTreeRegressor
from river import evaluate
from river import metrics

In [11]:
tree = HoeffdingTreeRegressor() # Versão da Hoeffding Tree para tarefas de regressão
rmse = metrics.RMSE() # A river tem classes prontas para boa parte das métricas comuns

In [16]:
# fetch dataset
wine = fetch_ucirepo(id=186)

# data (as pandas dataframes)
X = wine.data.features
y = wine.data.targets["quality"]

`learn_one` - método para ajustar vendo apenas 1 única instância por vez
<br>
`predict_one` - métodos para previsão de 1 único valor target

In [13]:
for x,yi in stream.iter_pandas(X, y):
  # Este é o framework comum de validação em fluxo de dados
  # Test-then-train
  # Para evitar vazamento, primeiro o modelo faz a predição e atualiza o erro
  # Por fim, o modelo ajusta os parâmetros usando essa instância

  # 1. Teste
  y_pred = tree.predict_one(x)
  # 2. Atualiza métrica
  rmse.update(yi, y_pred)
  # 3. Ajuste
  tree.learn_one(x, yi)

In [14]:
print(rmse)

RMSE: 3.406538
